In [1]:
from numpy import mean
from numpy import std
from numpy import dstack
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Conv1D
from tensorflow.keras.layers import MaxPooling1D
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import LSTM
import numpy as np
from sklearn.model_selection import train_test_split


In [2]:
s25 = np.load('s25.npy')
h25 = np.load('h25.npy')

In [3]:
s25_labels = np.array([0 for _ in range(0,len(s25))])
h25_labels = np.array([1 for _ in range(0,len(h25))])

In [4]:
X = np.append(s25,h25,axis=0)

In [5]:
y = np.append(s25_labels,h25_labels,axis=0)

In [9]:
def evaluate_model(trainX, trainy, testX, testy):
    verbose, epochs, batch_size = 1, 15, 16
    n_timesteps, n_features = trainX.shape[1], trainX.shape[2]
    model = Sequential()
    model.add(LSTM(100, input_shape=(n_timesteps,n_features)))
    model.add(Dropout(0.5))
    model.add(Dense(100, activation= 'relu'))
    model.add(Dropout(0.25))
    model.add(Dense(1, activation= 'sigmoid' ))
    model.compile(loss= 'binary_crossentropy' , optimizer= 'adam' , metrics=['accuracy'])
    # fit network
    model.fit(trainX, trainy, epochs=epochs, batch_size=batch_size, verbose=verbose)
    # evaluate model
    _, accuracy = model.evaluate(testX, testy, batch_size=batch_size, verbose=0)
    return accuracy

In [10]:
def summarize_results(scores):
    print(scores)
    m, s = mean(scores), std(scores)
    print( ' Accuracy: %.3f%% (+/-%.3f) ' % (m, s))

In [11]:
def run_experiment(X,y,repeats=5):
    # load data
    trainX, testX,trainy, testy = train_test_split(X, y, test_size=0.20, random_state=42)
    # repeat experiment
    scores = list()
    for r in range(repeats):
        score = evaluate_model(trainX, trainy, testX, testy)
        score = score * 100.0
        print( ' >#%d: %.3f ' % (r+1, score))
        scores.append(score)
    # summarize results
    summarize_results(scores)

In [11]:
run_experiment(X,y)

Epoch 1/15
58/58 [==============================] - 10s 174ms/step - loss: 0.7014 - accuracy: 0.5586
Epoch 2/15
58/58 [==============================] - 10s 171ms/step - loss: 0.6069 - accuracy: 0.6659
Epoch 3/15
58/58 [==============================] - 10s 173ms/step - loss: 0.5520 - accuracy: 0.6977
Epoch 4/15
58/58 [==============================] - 10s 173ms/step - loss: 0.4927 - accuracy: 0.7722
Epoch 5/15
58/58 [==============================] - 10s 174ms/step - loss: 0.4082 - accuracy: 0.8171
Epoch 6/15
58/58 [==============================] - 10s 174ms/step - loss: 0.3393 - accuracy: 0.8423
Epoch 7/15
58/58 [==============================] - 10s 174ms/step - loss: 0.2803 - accuracy: 0.8795
Epoch 8/15
58/58 [==============================] - 10s 172ms/step - loss: 0.2292 - accuracy: 0.9091
Epoch 9/15
58/58 [==============================] - 10s 175ms/step - loss: 0.2076 - accuracy: 0.9168
Epoch 10/15
58/58 [==============================] - 10s 172ms/step - loss: 0.1805 - accura

In [6]:
def evaluate_model_2(trainX, trainy, testX, testy):
    verbose, epochs, batch_size = 1, 15, 16
    n_timesteps, n_features = trainX.shape[1], trainX.shape[2]
    model = Sequential()
    model.add(LSTM(100, return_sequences=True, input_shape=(n_timesteps,n_features)))
    model.add(LSTM(50))
    model.add(Dropout(0.5))
    model.add(Dense(100, activation= 'relu'))
    model.add(Dropout(0.25))
    model.add(Dense(1, activation= 'sigmoid' ))
    model.compile(loss= 'binary_crossentropy' , optimizer= 'adam' , metrics=['accuracy'])
    # fit network
    model.fit(trainX, trainy, epochs=epochs, batch_size=batch_size, verbose=verbose)
    # evaluate model
    _, accuracy = model.evaluate(testX, testy, batch_size=batch_size, verbose=0)
    return accuracy

In [7]:
def run_experiment_2(X,y,repeats=5):
    # load data
    trainX, testX,trainy, testy = train_test_split(X, y, test_size=0.20, random_state=42)
    # repeat experiment
    scores = list()
    for r in range(repeats):
        score = evaluate_model_2(trainX, trainy, testX, testy)
        score = score * 100.0
        print( ' >#%d: %.3f ' % (r+1, score))
        scores.append(score)
    # summarize results
    summarize_results(scores)

In [ ]:
run_experiment_2(X,y)

Epoch 1/15
58/58 [==============================] - 17s 287ms/step - loss: 0.6446 - accuracy: 0.6057
Epoch 2/15
58/58 [==============================] - 17s 287ms/step - loss: 0.5039 - accuracy: 0.7579
Epoch 3/15
58/58 [==============================] - 17s 286ms/step - loss: 0.3689 - accuracy: 0.8445
Epoch 4/15
58/58 [==============================] - 17s 290ms/step - loss: 0.2988 - accuracy: 0.8817
Epoch 5/15
58/58 [==============================] - 17s 288ms/step - loss: 0.2394 - accuracy: 0.9124
Epoch 6/15
58/58 [==============================] - 17s 287ms/step - loss: 0.1938 - accuracy: 0.9277
Epoch 7/15
58/58 [==============================] - 17s 289ms/step - loss: 0.1409 - accuracy: 0.9518
Epoch 8/15
58/58 [==============================] - 17s 288ms/step - loss: 0.2224 - accuracy: 0.9211
Epoch 9/15
58/58 [==============================] - 17s 290ms/step - loss: 0.1573 - accuracy: 0.9463
Epoch 10/15
58/58 [==============================] - 17s 287ms/step - loss: 0.2427 - accura